In [1]:
import os
import pandas as pd
import time
import subprocess
import multiprocessing
import math
from Bio import Entrez
from Bio import SeqIO
from Bio import motifs
from tqdm import tqdm
import warnings

def extra_upORF(seq_record, upORFs = '', uplength = 20, downlength = 0): #提取启动子序列
    if uplength + downlength <= 0:
        raise ValueError("The length of the upORF cannot be zero or negative.")
        return upORFs
    if 'circular' in seq_record.annotations['topology']:
        for feature in seq_record.features:
            if feature.type == 'CDS':
                if feature.location.strand == 1:
                    if (feature.location.end - feature.location.start) == len(seq_record.seq) and len(feature.location.parts) >= 2:
                        start_index = feature.location.parts[0].start
                    else:
                        start_index = feature.location.start
                    if (start_index - uplength) < 0:
                        start_index = start_index + len(seq_record.seq)
                        upORF_seq = (seq_record.seq+seq_record.seq)[(start_index - uplength):(start_index + downlength)]
                    elif (start_index + downlength) >= len(seq_record.seq):
                        upORF_seq = (seq_record.seq+seq_record.seq)[(start_index - uplength):(start_index + downlength)]
                    else:
                        upORF_seq = seq_record.seq[(start_index - uplength):(start_index + downlength)]
                else:
                    if (feature.location.end - feature.location.start) == len(seq_record.seq) and len(feature.location.parts) >= 2:
                        end_index = feature.location.parts[0].end
                    else:
                        end_index = feature.location.end
                    if (end_index + uplength) >= len(seq_record.seq):
                        upORF_seq = (seq_record.seq+seq_record.seq)[(end_index - downlength):(end_index + uplength)].reverse_complement()
                    elif (end_index - downlength) < 0:
                        end_index = end_index + len(seq_record.seq)
                        upORF_seq = (seq_record.seq+seq_record.seq)[(end_index - downlength):(end_index + uplength)].reverse_complement()
                    else:
                        upORF_seq = seq_record.seq[(end_index - downlength):(end_index + uplength)].reverse_complement()
                if len(upORF_seq) < (uplength + downlength):
                    continue
                elif upORF_seq.strip('ATCG') != '':
                    continue
                else:
                    try:
                        upORFs += '>' + feature.qualifiers['locus_tag'][0] + '\n' + str(upORF_seq) + '\n'
                    except:
                        upORFs += '>' + feature.qualifiers['protein_id'][0] + '\n' + str(upORF_seq) + '\n'
        return upORFs
    else:
        for feature in seq_record.features:
            if feature.type == 'CDS':
                if feature.location.strand == 1:
                    upORF_seq = seq_record.seq[max(0,feature.location.start - uplength):max(0,feature.location.start + downlength)]
                else:
                    upORF_seq = seq_record.seq[max(0,feature.location.end - downlength):(feature.location.end + uplength)].reverse_complement()
                if len(upORF_seq) < (uplength + downlength):
                    continue
                elif upORF_seq.strip('ATCG') != '':
                    continue
                else:
                    try:
                        upORFs += '>' + feature.qualifiers['locus_tag'][0] + '\n' + str(upORF_seq) + '\n'
                    except:
                        upORFs += '>' + feature.qualifiers['protein_id'][0] + '\n' + str(upORF_seq) + '\n'
        return upORFs

def notATCG(str):
    atcg = ['A', 'T', 'C', 'G']
    for i in str:
        if i in atcg:
            continue
        else: 
            return True
    return False

def run_motif_finder(assembly, que):
    os.chdir(f'/home/zhongshitong/data/up-ORF-motif_local/assembly/{assembly}')
    seq_data = SeqIO.parse(f'/data/zhongshitong/Bacteria_reference_genome/ncbi_dataset/data/{assembly}/genomic.gbff', format="gb")
    upORF_file = open('upORFseqs.txt', 'w+')
    upORFs = ''
    CDS_count = 0
    for seq_record in seq_data:
        upORFs += extra_upORF(seq_record)

    upORF_file.write(upORFs)
    upORF_file.close()

    os.system('nohup meme upORFseqs.txt -dna -w 10 -mod zoops -nmotifs 3 -brief 50000 -nostatus -p 1 -nostatus > /dev/null 2>&1')
    que.put(1)

def pbar_fun(que, tot):
    count = 0
    with tqdm(total = tot, desc='Program', leave=True, ncols=100, unit='B', unit_scale=True) as pbar:
        while True:
            if not que.empty():
                value = que.get(True)
                count += 1
                pbar.update(1)
                if count == tot:
                    break
            else:
                continue

warnings.filterwarnings('ignore')

Entrez.email = 'zhongshitong@zju.edu.cn'

os.chdir('/home/zhongshitong/data/up-ORF-motif_local')

data = pd.read_csv('ncbi_genome-reference-annotation-complete-Ex_Cand_sym-20241211.csv')

In [2]:
manager = multiprocessing.Manager()
que = manager.Queue()
tax = []

par = 33
tot = len(data['Assembly Accession'])
pool = multiprocessing.Pool(par)
pool.apply_async(pbar_fun,args=(que, tot))

for assembly in data['Assembly Accession']:
    try: os.makedirs(f'/home/zhongshitong/data/up-ORF-motif_local/assembly/{assembly}')
    except: pass
    pool.apply_async(run_motif_finder,(assembly, que))
    seq_data = SeqIO.parse(f'/data/zhongshitong/Bacteria_reference_genome/ncbi_dataset/data/{assembly}/genomic.gbff', format="gb")
    for seq_record in seq_data:
        break
    org = seq_record.annotations['organism']
    if org.split(' ')[1] in seq_record.annotations['taxonomy'][-1]:
        taxo = seq_record.annotations['taxonomy'][:-1].copy()
    else:
        taxo = seq_record.annotations['taxonomy'].copy()
    taxo.insert(0,assembly)
    taxo.append(org)
    file = open(f'/home/zhongshitong/data/up-ORF-motif_local/assembly/{assembly}/assembly info.txt', 'w+')
    file.write(";".join(taxo))
    file.close()

pool.close()
pool.join()

Program: 100%|████████████████████████████████████████████████| 5.01k/5.01k [4:21:35<00:00, 3.13s/B]


In [3]:
with tqdm(total = len(data['Assembly Accession']), desc='Program', leave=True, ncols=100, unit='B', unit_scale=True) as pbar:
    table = pd.DataFrame(columns=["Assembly", "motif_consensus", "motif_degenerate_consensus", "motif_instances_number", "evalue", "Assembly_records_number"])
    non_motif_assembly = []
    motifs_data = ''
    motifs_fasta = ''
    count = 0
    ex = []
    for i in range(len(data['Assembly Accession'])):
        assembly = data['Assembly Accession'][i]
        if not pd.isnull(assembly):
            pass
        else:
            assembly = data['GeneBank'][i]
        pbar.update(1)
        os.chdir(f'/home/zhongshitong/data/up-ORF-motif_local/assembly/{assembly}')
        
        try:
            handle = open("meme_out/meme.xml")
            record = motifs.parse(handle, "meme")
        except:
            print(assembly, i)
            ex.append(assembly)
            continue
        match = False
        for motif in record:
            if motif.evalue < 0.01 and len(motif.alignment.sequences) >= 20 and notATCG(str(motif.degenerate_consensus)):
                match = True
                temptable = pd.DataFrame([[assembly, str(motif.consensus), str(motif.degenerate_consensus), len(motif.alignment.sequences), motif.evalue, len(record.sequences)]], columns=["Assembly", "motif_consensus", "motif_degenerate_consensus", "motif_instances_number", "evalue", "Assembly_records_number"])
                table = pd.concat([table, temptable])
                motif.name = assembly.replace(".", "_") + '_' + motif.id
                motifs_data += format(motif, "jaspar").replace("None ", "")
                motifs_fasta = motifs_fasta + '>' + motif.name + '\n' + str(motif.degenerate_consensus) + '\n'
                count += 1
                continue
            elif len(motif.alignment.sequences) > 150 and notATCG(str(motif.degenerate_consensus)):
                match = True
                temptable = pd.DataFrame([[assembly, str(motif.consensus), str(motif.degenerate_consensus), len(motif.alignment.sequences), motif.evalue, len(record.sequences)]], columns=["Assembly", "motif_consensus", "motif_degenerate_consensus", "motif_instances_number", "evalue", "Assembly_records_number"])
                table = pd.concat([table, temptable])
                motif.name = assembly.replace(".", "_") + '_' + motif.id
                motifs_data += format(motif, "jaspar").replace("None ", "")
                motifs_fasta = motifs_fasta + '>' + motif.name + '\n' + str(motif.degenerate_consensus) + '\n'
                count += 1
                continue
            elif motif.evalue > 1 or len(motif.alignment.sequences) < 20:
                break
        if match:
            pass
        else:
            non_motif_assembly.append(assembly)

os.chdir('/home/zhongshitong/data/up-ORF-motif_local')
motifs_file = open('motifs_data.txt', 'w+')
motifs_file.write(motifs_data)
motifs_file.close()
motifs_file = open('motifs_data.fasta', 'w+')
motifs_file.write(motifs_fasta)
motifs_file.close()
# 输出motif基本信息
table = table.reset_index(drop=True)
table.to_csv('up-ORF-motif.csv')
#print(non_motif_assembly)
print('motif count:', count)
print(ex)

Program: 100%|██████████████████████████████████████████████████| 5.01k/5.01k [10:45<00:00, 7.76B/s]

motif count: 6787
[]


In [4]:
# 画motif系统树

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
import pandas as pd
import os
from tqdm import tqdm
import multiprocessing
from multiprocessing import Process, Queue, Pool


def motif_align(i, num, que):
    results = []
    for j in range(i+1, num+1):
        robjects.r(f'd{i}_{j} <- matalign(list(pcms[[{i}]], pcms[[{j}]]),revcomp=FALSE,)')
        with (robjects.default_converter + pandas2ri.converter).context():
            pd_from_r_df = robjects.conversion.get_conversion().rpy2py(robjects.r(f'd{i}_{j}'))
        robjects.r(f'rm(d{i}_{j})')
        row = pd_from_r_df['motif1'][0]
        col = pd_from_r_df['motif2'][0]
        dis = pd_from_r_df['distance'][0]
        result = [row, col, dis]
        results.append(result.copy())
    que.put(results)
    
def pbar_fun_m(que, que_o, num, par):
    count = 0
    results = []
    with tqdm(total = num, desc='Program', leave=True, ncols=100, unit='B', unit_scale=True) as pbar:
        while True:
            if not que.empty():
                result = que.get(True)
                results = result.copy()
                count += 1
                que_o.put(results)
                pbar.update(1)
                if count == num:
                    break
            else:
                continue

os.chdir("/home/zhongshitong/data/up-ORF-motif_local")
robjects.r('library("motifStack")')
robjects.r('library("ape")')
robjects.r('setwd("/home/zhongshitong/data/up-ORF-motif_local")')
robjects.r('pcms<-importMatrix("motifs_data.txt", format="jaspar", to="pcm")')
num = int(str(robjects.r('length(pcms)')).split(' ')[1])
robjects.r(f'd <- matalign(list(pcms[[1]], pcms[[2]]),revcomp=FALSE,)')
with (robjects.default_converter + pandas2ri.converter).context():
    r_df = robjects.conversion.get_conversion().rpy2py(robjects.r('d'))
total_data = pd.DataFrame(columns=r_df.columns)
robjects.r(f'motifs <- unique(names(pcms[1:{num}]))')
robjects.r('da <- matrix(NA, nrow = length(motifs), ncol = length(motifs))')
robjects.r('rownames(da) <- colnames(da) <- motifs')

manager = multiprocessing.Manager()
que = manager.Queue()
que_o = manager.Queue()

par = 32+1
pool = multiprocessing.Pool(par)
pool.apply_async(pbar_fun_m,args=(que, que_o, num, par))

for i in range(1, num+1):
    pool.apply_async(motif_align,(i, num, que))
    
pool.close()
count = 0
with tqdm(total = num*(num-1)/2, desc='Program', leave=True, ncols=100, unit='B', unit_scale=True) as pbar:
    while True:
        if not que_o.empty():
            results = que_o.get(True)
            for result in results:
                row = result[0]
                col = result[1]
                dis = result[2]
                robjects.r(f'da["{row}", "{col}"] <- {dis}')
                robjects.r(f'da["{col}", "{row}"] <- {dis}')
                count += 1
                pbar.update(1)
            if count == num*(num-1)/2:
                break
        else:
            continue
pool.join()

robjects.r('write.csv(da, file = "motif_align_result.csv",)')
robjects.r('dan <- dist(da)') #借用距离函数，但不直接用距离生成系统树
with tqdm(total = num*(num-1)/2, desc='Program', leave=True, ncols=100, unit='B', unit_scale=True) as pbar:
    count = 1
    for i in range(num):
        for j in range(i+1, num):
            robjects.r(f'dan[{count}] <- da[{i+1}, {j+1}]') #更换距离函数得到的数据，改为两个motif之间的比对距离
            count += 1
            pbar.update(1)
robjects.r('hc <- do.call("hclust", list(dan, method="single"))')
robjects.r('tr <- as.phylo(hc)')
robjects.r('write.tree(tr, file = "tr_single.txt",)')

R[write to console]: 载入需要的程序包：grid

R[write to console]: 

Program: 100%|████████████████████████████████████████████████| 6.79k/6.79k [1:59:02<00:00, 1.05s/B]
Program: 100%|███████████████████████████████████████████████| 23.0M/23.0M [5:40:06<00:00, 1.13kB/s]
Program: 100%|███████████████████████████████████████████████| 23.0M/23.0M [2:04:29<00:00, 3.08kB/s]


In [ ]:
# 重读文件重新画树，可跳过
robjects.r('da <- read.csv(file = "motif_align_result.csv", row.names = 1, header=T)')
robjects.r('dan <- dist(da)')
with tqdm(total = num*(num-1)/2, desc='Program', leave=True, ncols=100, unit='B', unit_scale=True) as pbar:
    count = 1
    for i in range(num):
        for j in range(i+1, num):
            robjects.r(f'dan[{count}] <- da[{i+1}, {j+1}]')
            count += 1
            pbar.update(1)
robjects.r('hc <- do.call("hclust", list(dan, method="single"))')
robjects.r('tr <- as.phylo(hc)')
robjects.r('write.tree(tr, file = "trn_single.txt",)')

In [5]:
from Bio import Phylo

os.chdir("/home/zhongshitong/data/up-ORF-motif_local")
tree = Phylo.read("tr_single.txt", "newick")
motifs_data = tree.get_terminals()

def motif_indexing(motif_n, seq_nu = 10):
    ass_dir = motif_n.split('_')[0]+'_'+motif_n.split('_')[1]+'.'+motif_n.split('_')[2]

    os.chdir(f"/home/zhongshitong/data/up-ORF-motif_local/assembly/{ass_dir}")

    handle = open("meme_out/meme.xml")
    record = motifs.parse(handle, "meme")
    seq_res = SeqIO.parse("upORFseqs.txt","fasta")
    upORF_info = ''
    for motif in record:
        if motif.id != motif_n.split('_')[-2]+'_'+motif_n.split('_')[-1]:
            continue
        count = 0
        for seq in seq_res:
            for i in range(seq_nu):
                if motif.instances[i].sequence_name == seq.id:
                    upORF_info += '>'+ass_dir+'-'+seq.id+'\n'+seq.seq+'\n'
                    count += 1
            if count == seq_nu:
                break
    return str(upORF_info)

motif_clu = {'blue_AG': ['GCF_000226315_1_motif_1', 'GCF_007859615_1_motif_2'], 
             'red_TA': ['GCF_018406645_1_motif_2', 'GCF_030408795_1_motif_3'],
             'green_TA': ['GCF_018289135_1_motif_1','GCF_006542645_1_motif_1'],}

for item in motif_clu:
    try: os.makedirs(f'/home/zhongshitong/data/up-ORF-motif_local/filter-results/{item}')
    except: pass
    upORF_rec = ''
    filt_re = tree.common_ancestor(motif_clu[item][0], motif_clu[item][1]).get_terminals()
    print(item)
    with tqdm(total = len(filt_re), desc='Program', leave=True, ncols=100, unit='B', unit_scale=True) as pbar:
        for f_re in filt_re:
            upORF_rec += motif_indexing(str(f_re))
            pbar.update(1)

    os.chdir(f'/home/zhongshitong/data/up-ORF-motif_local/filter-results/{item}')
    upORF_file = open(f"upORFseqs.txt", "w+")
    upORF_file.write(upORF_rec)
    upORF_file.close()
    os.system('nohup meme upORFseqs.txt -dna -w 10 -mod oops -nmotifs 1 -brief 60000 -nostatus -p 1 -nostatus > /dev/null 2>&1')
    
TA_motif_clu = {'yellow': ['GCF_008329925_1_motif_2', 'GCF_002222655_1_motif_2'], 
                 'pink': ['GCF_013003965_1_motif_2', 'GCF_003574315_2_motif_2'],
                 'purple': ['GCF_000950575_1_motif_2','GCF_014770185_1_motif_2'],
               'orange': ['GCF_035231185_1_motif_3','GCF_030408795_1_motif_3'],}

for item in TA_motif_clu:
    try: os.makedirs(f'/home/zhongshitong/data/up-ORF-motif_local/filter-results/red_TA_clu/{item}')
    except: pass
    upORF_rec = ''
    filt_re = tree.common_ancestor(TA_motif_clu[item][0], TA_motif_clu[item][1]).get_terminals()
    print(item)
    with tqdm(total = len(filt_re), desc='Program', leave=True, ncols=100, unit='B', unit_scale=True) as pbar:
        for f_re in filt_re:
            upORF_rec += motif_indexing(str(f_re))
            pbar.update(1)

    os.chdir(f'/home/zhongshitong/data/up-ORF-motif_local/filter-results/red_TA_clu/{item}')
    upORF_file = open(f"upORFseqs.txt", "w+")
    upORF_file.write(upORF_rec)
    upORF_file.close()
    os.system('nohup meme upORFseqs.txt -dna -w 10 -mod oops -nmotifs 1 -brief 60000 -nostatus -p 1 -nostatus > /dev/null 2>&1')

blue_AG


Program: 100%|██████████████████████████████████████████████████| 4.91k/4.91k [19:32<00:00, 4.18B/s]


red_TA


Program: 100%|██████████████████████████████████████████████████| 1.15k/1.15k [04:03<00:00, 4.71B/s]


green_TA


...

Program: 100%|████████████████████████████████████████████████████| 90.0/90.0 [00:09<00:00, 9.06B/s]


purple


Program: 100%|██████████████████████████████████████████████████████| 844/844 [02:49<00:00, 4.97B/s]


orange


Program: 100%|██████████████████████████████████████████████████████| 147/147 [00:25<00:00, 5.74B/s]


In [6]:
# 物种信息的汇总

os.chdir('/home/zhongshitong/data/up-ORF-motif_local')
data = pd.read_csv('ncbi_genome-reference-annotation-complete-Ex_Cand_sym-20241211.csv')
Ass_data = pd.DataFrame(columns=["Assembly accession", "Phylum", "TA-ORF proportion"])

TA_motifs = tree.common_ancestor(motif_clu['red_TA'][0], motif_clu['red_TA'][1]).get_terminals() + tree.common_ancestor(motif_clu['green_TA'][0], motif_clu['green_TA'][1]).get_terminals()
TA_motifs_n = []
for item in TA_motifs:
    TA_motifs_n.append(str(item))
    
with tqdm(total = len(data['Assembly Accession']), desc='Program', leave=True, ncols=100, unit='B', unit_scale=True) as pbar:
    for assembly in data['Assembly Accession']:
        upORFseqs = [seq.id for seq in SeqIO.parse(f"/home/zhongshitong/data/up-ORF-motif_local/assembly/{assembly}/upORFseqs.txt","fasta")]
        ORFcount = len(upORFseqs)
        handle = open(f"/home/zhongshitong/data/up-ORF-motif_local/assembly/{assembly}/meme_out/meme.xml")
        record = motifs.parse(handle, "meme")
        TA_ORFcount = 0
        for motif in record:
            motif_n = assembly.replace(".", "_") + '_' + motif.id
            if motif_n in TA_motifs_n:
                TA_ORFcount += len(motif.alignment.sequences)
        TAprop = TA_ORFcount/ORFcount
        file = open(f'/home/zhongshitong/data/up-ORF-motif_local/assembly/{assembly}/assembly info.txt', 'r')
        phy = ''
        for line in file:
            for word in line.split(';'):
                if word[-3:] == 'ota':
                    phy = word
                    break
        file.close()
        if phy == '':
            handle=Entrez.esearch(db='Assembly',term=assembly) #查找某一物种的基因组，一般返回参考基因组
            records=Entrez.read(handle)
            pmids=records["IdList"][0]
            handle = Entrez.elink(dbfrom="genome", db="nucleotide", LinkName="assembly_nuccore_refseq", id=pmids) #匹配参考基因组的具体核酸id，匹配RefSeq数据
            records=Entrez.read(handle)
            seq_data = records[0]['LinkSetDb'][0]['Link']
            for ids in seq_data:
                nuc_id = ids['Id']
                handle = Entrez.efetch(db="nucleotide", rettype="gbwithparts", retmode="text", id=nuc_id)
                seq_record = SeqIO.read(handle, 'genbank')
                for word in seq_record.annotations['taxonomy']:
                    if word[-3:] == 'ota':
                        phy = word
                        break
        if phy == '':
            phy = 'unknown-' + line.split(';')[2]
        Ass_data = pd.concat([Ass_data, pd.DataFrame([{"Assembly accession": assembly, "Phylum": phy, "TA-ORF proportion": TAprop}])], ignore_index=True)
        pbar.update(1)
        
os.chdir('/home/zhongshitong/data/up-ORF-motif_local')
Ass_data.to_csv('Ass_data.csv')

Program: 100%|██████████████████████████████████████████████████| 5.01k/5.01k [14:51<00:00, 5.62B/s]
